In [4]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import csv
import collections
import pandas as pd
import math
import json


In [5]:
#load grid 
# This is the grid object, which is used throughout all data preprocessing.
# It represents the city of Austin through a series of grids.
# It thus makes a tractable way to compute distance between grids, ect. 
class Grid():
    def __init__(self, grid_json):
        self.grid = grid_json
        self.min_lat = self.grid["latitude_min"]
        self.min_lon = self.grid["longitude_min"]
        self.max_lat = self.grid["latitude_max"]
        self.max_lon = self.grid["longitude_max"]
        self.latitude_delta = self.grid["latitude_step"]
        self.longitude_delta = self.grid["longitude_step"]
        self.nrows = math.ceil((self.max_lat - self.min_lat) / self.latitude_delta)
        self.ncols = math.ceil((self.max_lon - self.min_lon) / self.longitude_delta)
        self.times = self.grid["time_matrix"]
        self.census_tract_region_map = self.grid["census_tract_region_mapping"]
        self.region_to_tract = collections.defaultdict(list)
        for census_tract in self.census_tract_region_map:
            for region in self.census_tract_region_map[census_tract]:
                self.region_to_tract[region].append(census_tract)
    def map_point_to_region(self, latitude, longitude):
        return math.floor((latitude-self.min_lat)/self.latitude_delta) * self.ncols  + math.floor((longitude-self.min_lon)/self.longitude_delta)
    def get_representative(self, region_num):
        row_num = region_num//self.ncols
        col_num = region_num - row_num*self.ncols
        lat = self.min_lat + row_num * self.latitude_delta + 0.5*self.latitude_delta
        lon = self.min_lon + col_num * self.longitude_delta + 0.5*self.longitude_delta
        return [lon, lat]
    def get_time(self, region1, region2):
        try:
            return self.times[region1][region2]
        except IndexError:
            return -1
    def region_to_census_tract(self, region):
        try:
            return self.region_to_tract[region]
        except KeyError:
            return "0_0"

In [6]:
# Using smaller distance matrix for hopefully faster runtime of Julia code
with open("../Input_Data/grid_info_multiple.json", "r") as f:
    grid_json = json.load(f)
g = Grid(grid_json)

In [23]:
def writeNeighbors(reg_num, numrows, numcols):
    reg_idx = reg_num
    neighbors = []
    
    #write at reg_num
#     if (reg_num>=1) and (reg_num<=numrows*numcols):
#         neighbors.append(reg_idx)
    
    #row-1, col
    if not (reg_num//numcols == 0):
        neighbors.append(reg_idx-numcols)
    
    #row+1, col
    if  (reg_num <= (numrows-1)*numcols):
        neighbors.append(reg_idx+numcols)
    
    #row, col-1
    if not (reg_num%numcols == 1):
        neighbors.append(reg_idx-1)
    
    #row, col+1
    if  not (reg_num%numcols == 0):
        neighbors.append(reg_idx+1) 
    return neighbors

In [48]:
g.times[1][2]

0

In [51]:
#23k ->0
none_count = 0
for i in range(len(g.times)):
    for j in range(len(g.times)):
        if(g.get_time(i,j) == None):
            g.times[i][j] = 0
            none_count = none_count + 1
none_count

0

4023530.5700000003

In [90]:
def selfTime(regionNumber):
    nbhs = writeNeighbors(regionNumber,g.nrows,g.ncols)
    mean_time = 0
    for i in range(len(nbhs)):
        #print(a[i])
        #print(g.get_time(regionNumber,nbhs[i]))
        mean_time = mean_time + g.get_time(regionNumber,nbhs[i])
    mean_time = mean_time / len(nbhs)
    return mean_time

762.2
409.66
246.39
80.66


374.7275

In [85]:
selfTime(1000)

762.2
409.66
246.39
80.66


374.7275

In [91]:
#This is setting the time from region i to i. By default we set it to 5 minutes.
#selfTimeList = [] 
for i in range(len(g.times)):
    #selfTimeList.append(selfTime(i))
    g.times[i][i] = selfTime(i)

In [101]:
np.mean(selfTimeList)

326.60992135416666

In [97]:
import json
d = {"latitude_min": g.min_lat, "longitude_min": g.min_lon, "latitude_max": g.max_lat, "longitude_max": g.max_lon, "latitude_step": g.latitude_delta, "longitude_step": g.longitude_delta}
d["time_matrix"] = g.times
d["census_tract_region_mapping"] = g.grid["census_tract_region_mapping"]
d["NUM_REGIONS"] = 3200
# with open("grid_info_multiple.json", "w") as j:
s = "grid_info_" + str(3200) + ".json"

with open(s, "w") as j:
    json.dump(d, j)
    

In [98]:
# Using smaller distance matrix for hopefully faster runtime of Julia code
with open("../Input_Data/grid_info_3200_v2.json", "r") as f:
    grid_json = json.load(f)
g = Grid(grid_json)

In [100]:
g.get_time(1000,1001)

80.66